## 1º Definir função para simular o circuito e retornar os dados da corrente elétrica

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  4 21:21:44 2020

@author: Edson Porto da Silva
"""
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import signal

def simulador(R,L,C,a):
    
    # Resolução numérica de um circuito RLC série alimentado por uma onda quadrada
    # de tensão.


    # parâmetros do circuito:
    V0 = 0                # tensão inicial no capacitor [volts]
    I0 = 0                # corrente inicial no indutor [ampères]
    fq = 2               # frequência da onda quadrada [hertz]
    Vq = 15              # amplitude da onda quadrada [volts]
    w0 = 1/np.sqrt(L*C)   # frequência angular de ressonância [rad/segundo]

    t = np.arange(0,5,0.001)*(2/10)   # discretização do intervalo de tempo [segundos]
    Vs = Vq*signal.square(2*np.pi*fq*t) # onda quadrada na entrada do circuito

    vC    = np.zeros(len(t))
    x     = np.zeros(len(t))

    # EDO da tensão sobre o capacitor: vc''(t)+(R/L)vc'(t)+vc(t)/LC = vs/LC

    # Solução numérica:
    vC[0] = V0       # condição incial de vc
    x[0]  = I0/C     # condição inicial da derivada vc'(t)
    #
    # Integração numérica via método de Euler:
    deltaT    = t[1]-t[0] # passo de integração
    numPoints = len(t)-1

    for kk in range(0, numPoints):
        vC[kk+1] = vC[kk]+x[kk]*deltaT                                # calcula vc(t+deltaT)
        x[kk+1]  = x[kk]+(-R/L*x[kk]-1/(L*C)*(vC[kk]-Vs[kk]))*deltaT  # calcula vc'(t+deltaT)

    # cálculo das tensões e da corrente partir de vc(t):
    i  = np.append(I0, C*np.diff(vC)/deltaT)    # corrente no circuito
    
    vR = R*i                                    # tensão no resistor
    vL = Vs-vR-vC                               # tensão no indutor(LKT)
    if a ==1:
        
        plt.figure(1, figsize=(12, 9), dpi=80, facecolor='w', edgecolor='k')
        plt.plot(t, i)
        plt.legend(['corrente i(t)']);
        plt.grid(color='k', linestyle='--', linewidth=0.1)
        plt.ylabel('ampères (A)', fontsize = 14)
        plt.xlabel('tempo (s)', fontsize = 14)
        plt.title('Corrente no circuito RLC série', fontsize = 14)
        plt.show(block=False)
    return(i)

## 2º Gerar os valores de R,L e C

In [ ]:

conjunto = np.empty((0,3)) #Matriz que armazena os valores de R,L e C
tam = 0 #Quantidade de valores armazenados (linhas da matriz)
w_arm = np.empty(())
wd_arm = np.empty(())
while tam < 100:    #Superamortecido

    l = random.uniform(0.01, 100)
    c = random.uniform(1e-8, 1e-4)
    r = random.uniform(1, 1e4)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    param= abs(w-alpha)
    if alpha>w and w>10 and w<200 : 
        w_arm = np.hstack((w_arm,w))
        conj =r,l,c
        conjunto =np.vstack((conjunto,conj))
        tam+=1
      
while tam < 200:     #Subamortecido

    l = random.uniform(0.01, 100)
    c = random.uniform(1e-8, 1e-4)
    r = random.uniform(1, 1e4)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    if alpha<w and w>10 and w<200  :
        wd = ((w**2) - (alpha**2))**(1/2)
        if  wd<3140:
            w_arm = np.hstack((w_arm,w))
            wd_arm = np.hstack((wd_arm,wd))
            conj =r,l,c
            conjunto =np.vstack((conjunto,conj))
            tam+=1
while tam < 300:     #Criticamente amortecido
    
    l = random.uniform(0.01, 100)
    c = random.uniform(1e-8, 1e-4)
    r = random.uniform(1, 1e4)
    w = 1/(l*c)**(1/2)
    alpha = r/(2*l)
    param= abs(w-alpha)
    if param<0.001 and w>10 and w<200 :
        w_arm = np.hstack((w_arm,w))
        conj =r,l,c
        conjunto =np.vstack((conjunto,conj))
        tam+=1



### 3º Avaliar os dados 

In [ ]:
print("Valor máximo de w: ",np.max(w_arm))
print("Valor mínimo de w: ",np.min(w_arm))
print("Valor médio de w: ",np.mean(w_arm))

print("")
print("Valor máximo de wd: ",np.max(wd_arm))
print("Valor mínimo de wd: ",np.min(wd_arm))
print("Valor médio de wd: ",np.mean(wd_arm))

print("")
print("Valor máximo de resistor: ",np.max(conjunto[:,0]))
print("Valor mínimo de resistor: ",np.min(conjunto[:,0]))
print("Valor médio de resistor: ",np.mean(conjunto[:,0]))

print("")
print("Valor máximo de indutor: ",np.max(conjunto[:,1]))
print("Valor mínimo de indutor: ",np.min(conjunto[:,1]))
print("Valor médio de indutor: ",np.mean(conjunto[:,1]))

print("")
print("Valor máximo de capacitor: ",np.max(conjunto[:,2]))
print("Valor mínimo de capacitor: ",np.min(conjunto[:,2]))
print("Valor médio de capacitor: ",np.mean(conjunto[:,2]))


## 4º Armazenar os valores de corrente gerados pelo conjunto de valores R,L,C

In [ ]:
m = conjunto.shape[0]
iresult=np.empty((0,5000))
for i in range (m):
    R=conjunto[i,0]
    L=conjunto[i,1]
    C=conjunto[i,2]
    print("Gráfico número:",i,"\n")
    it = simulador(R,L,C,1)
    iresult= np.vstack((iresult,it))

In [ ]:

# Obtém a média dos valores de entrada
media_iresult = np.mean(iresult)

# Obtém o desvio padrão dos valores de entrada
desvio_padrao_iresult = np.std(iresult)

# Normaliza os valores de entrada
iresult_norm = (iresult - media_iresult) / desvio_padrao_iresult


In [ ]:
import tensorflow as tf  #O tensorrflow é uma biblioteca especializada em machine learning
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout


#Esse é a arquitetura da nossa rede neural:
model_2 = Sequential ([
    Dense (units=5000, activation= 'relu'), #Nas camadas ocultas de uma rede neural é utilizado comumente a ativação Relu;
    Dense (units=3, activation='relu'), #A última camada é a de saída ela leva a ativação que reflete o problema
])
#O parâmetro "units" da primeira e última camada deve corresponder ao número de entradas e saídas do seu modelo respectivamente


#Agora importamos o nosso algoritmo de perdas, ou erros. Como uma cost function
from tensorflow.keras.losses import MeanSquaredError 

model_2.compile (optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
loss=MeanSquaredError(),  #Atribuimos a função de erro desejada ao modelo
metrics=['accuracy']) 

#.fit usa uma função semelhante, porém mais avançada que o gradient descent para treinar o modelo
#epochs é o número de iterações
model_2.fit(iresult,conjunto,epochs=30)
model_2.summary()

''''O algoritmo Adam otimiza a taxa de aprendizado do gradient descent.
Isto é: se é perceptível que podemos aumentar ou diminuir a taxa de aprendizado ele fará isso automaticamente
Adam: Adaptative Moment estimation'''



In [ ]:
test_loss, test_acc = model_2.evaluate(iresult,  conjunto, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
y_3=model_2.predict(iresult)

print(y_3)

In [ ]:
print(conjunto)

In [ ]:
simulador(5.80851611e+03,  2.14692688e+02,  8.00736904e+00,1)
simulador(5.92417385e+03, 3.95611159e+01, 8.71784259e-05,1)